In [31]:
import pandas as pd
import numpy as np
import sys
import os
import datetime as dt
sys.path.append('./Others/Data_Science/Project')

In [32]:
Temp2023 = pd.read_csv('Temp2023.csv', encoding='cp949')
Temp2023 = Temp2023[['일시','기온(°C)','강수량(mm)']]

In [33]:
# nan을 0으로 채우기
Temp2023['강수량(mm)'] = Temp2023['강수량(mm)'].fillna(0)

In [34]:
# 일시에서 시간을 따로 추출하여 행으로 추가함
# 일시는 날짜 성분만 남김
Temp2023['일시'] = pd.to_datetime(Temp2023['일시'])
hours_to_exclude = [1, 2, 3, 4]
Temp2023 = Temp2023[~Temp2023['일시'].dt.hour.isin(hours_to_exclude)]
Temp2023['시간'] = Temp2023['일시'].dt.hour
Temp2023['일시'] = Temp2023['일시'].dt.date


In [35]:
Temp2023 = Temp2023.drop(Temp2023.index[0])

# 시간이 0인 행들의 일시에서 하루 빼기
# 먼저 일시를 다시 datetime으로 변환 (현재 date 타입이므로)
Temp2023['일시'] = pd.to_datetime(Temp2023['일시'])

# 시간이 0인 행들의 일시에서 1일 빼기
mask = Temp2023['시간'] == 0
Temp2023.loc[mask, '일시'] = Temp2023.loc[mask, '일시'] - pd.Timedelta(days=1)

# 필요하다면 다시 date 타입으로 변환
Temp2023['일시'] = Temp2023['일시'].dt.date

# 인덱스 재설정 (선택사항)
Temp2023 = Temp2023.reset_index(drop=True)

In [36]:
sub2023 = pd.read_csv('sub2023.csv', encoding='cp949')

In [37]:
# 시청 먼저 해보기기
sub2023_cityhall = sub2023[(sub2023['역명']=='시청')]
sub2023_cityhall['수송일자'] = pd.to_datetime(sub2023_cityhall['수송일자'])

/var/folders/h8/5hxv73fd07d033x49543ghl00000gn/T/ipykernel_35422/4102320329.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub2023_cityhall['수송일자'] = pd.to_datetime(sub2023_cityhall['수송일자'])


In [38]:
# 새로운 데이터프레임 생성
# 시간을 새로운 column으으로 추가할 것임
passenger = sub2023_cityhall.iloc[:,6:].values.flatten()
hour = np.arange(0,len(passenger)) % 20 + 5
hour[hour == 24] = 0
sub2023df = pd.DataFrame({
    '일자': sub2023_cityhall['수송일자'].values.repeat(20),
    '시간': hour,
    '승하차구분' : sub2023_cityhall['승하차구분'].values.repeat(20),
    '호선' : sub2023_cityhall['호선'].values.repeat(20),
    '승차인원': passenger
})

In [39]:
# sub2023df의 일자를 date 타입으로 변환 (비교를 위해)
sub2023df['일자_date'] = sub2023df['일자'].dt.date

# 일자와 시간을 기준으로 left join
sub2023dfarr = sub2023df.merge(
    Temp2023,
    left_on=['일자_date', '시간'],
    right_on=['일시', '시간'],
    how='left'
)
sub2023dfarr = sub2023dfarr.drop(columns=['일자_date', '일시'])
sub2023dfarr['일자'] = pd.to_datetime(sub2023dfarr['일자'])

In [40]:
# 주말과 휴일 기입, 주말 = 1, 휴일 = 2, 주말인데 휴일인 날도 2로 처리
weekend = sub2023dfarr['일자'].dt.weekday >=5
sub2023dfarr['주말'] = 0
sub2023dfarr['주말'][weekend]=1
vacation = np.array([dt.date(2023,1,1), dt.date(2023,1,21), dt.date(2023,1,22), dt.date(2023,1,23),
                     dt.date(2023,1,24), dt.date(2023,3,1), dt.date(2023,5,5),dt.date(2023,5,27),
                     dt.date(2023,5,29), dt.date(2023,6,6), dt.date(2023,8,15),
                     dt.date(2023,9,28), dt.date(2023,9,29), dt.date(2023,9,30),
                      dt.date(2023,10,2), dt.date(2023,10,3),dt.date(2023,10,9),
                      dt.date(2023,12,25)])
sub2023dfarr['주말'][sub2023dfarr['일자'].dt.date.isin(vacation)] = 2 # 

/var/folders/h8/5hxv73fd07d033x49543ghl00000gn/T/ipykernel_35422/1041807346.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sub2023dfarr['주말'][weekend]=1
/var/folders/h8/5hxv73fd07d033x49543ghl00000gn/T/ipykernel_35422/1041807346.py:4: S

In [98]:
# ...existing code...
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 범주형 변수 인코딩
df_ml = sub2023dfarr.copy()
#df_ml['승차인원'] = np.log10(df_ml['승차인원']+1)  # 승차인원 로그 변환
df_ml['승하차구분'] = df_ml['승하차구분'].astype('category').cat.codes
df_ml['호선'] = df_ml['호선'].astype('category').cat.codes

# 입력 변수와 출력 변수 지정
X = df_ml[['시간', '승하차구분', '호선', '기온(°C)', '강수량(mm)', '주말']]
y = df_ml['승차인원']

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 회귀 모델 학습
model = RandomForestRegressor(n_estimators=20, random_state=42)
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
#mse = mean_squared_error(10**y_test-1, 10**y_pred-1)
print(f"테스트 MSE: {mse:.2f}")
# ...existing code...

테스트 MSE: 82204.05


In [99]:
np.sqrt(mse)

286.7124905873165

In [95]:
df_ml[(df_ml['일자']=='2023-01-17') & (df_ml['승하차구분']==1)]

,일자,시간,승하차구분,호선,승차인원,기온(°C),강수량(mm),주말
1300,2023-01-17,5,1,0,2.155336,-6.6,0.0,0
1301,2023-01-17,6,1,0,2.934498,-6.7,0.0,0
1302,2023-01-17,7,1,0,3.498173,-6.8,0.0,0
1303,2023-01-17,8,1,0,3.960328,-6.8,0.0,0
1304,2023-01-17,9,1,0,3.571709,-6.4,0.0,0
1305,2023-01-17,10,1,0,3.205746,-5.0,0.0,0
1306,2023-01-17,11,1,0,3.164947,-3.8,0.0,0
1307,2023-01-17,12,1,0,3.064458,-3.1,0.0,0
1308,2023-01-17,13,1,0,3.054230,-1.7,0.0,0
1309,2023-01-17,14,1,0,2.977724,-0.5,0.0,0


In [96]:
new_X = pd.DataFrame([[8, 1, 0, 0.9, 0, 0]], columns=['시간', '승하차구분', '호선', '기온(°C)', '강수량(mm)', '주말'])
pred_y = model.predict(new_X)
print(f"예측된 승차인원: {10**pred_y[0]+1:.0f}")  # 로그 변환을 되돌림
print(f"예측된 승차인원: {pred_y[0]:.0f}") 

예측된 승차인원: 8890
예측된 승차인원: 4
